<a href="https://colab.research.google.com/github/Doaa-ali790/Doaa-ali790/blob/main/Another_copy_of_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
من  google.colab  استيراد  محرك الأقراص
drive.mount ( '/content/drive' )

project_dir =  '/content/drive/MyDrive/species_project'
print ( "دليل المشروع:" ،  project_dir )

SyntaxError: invalid character '،' (U+060C) (ipython-input-1732467623.py, line 5)

In [ ]:
import os
print(os.listdir(project_dir))

['train_features_and_svm.py', 'app_streamlit.py', 'requirements.txt', 'README.txt', 'svc_pipeline.joblib', 'dataset']


In [ ]:
# ✅ 1️⃣ ربط Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2️⃣ تثبيت مكتبة kaggle لتحميل البيانات (مرة واحدة فقط)
!pip install -q kaggle

# ✅ 3️⃣ إعداد مفتاح Kaggle (تضعيه لمرة واحدة فقط)
# انسخي مفتاحك من موقع كاجل وضعيه هنا
import os
os.environ['KAGGLE_USERNAME'] = "doaaali790"
os.environ['KAGGLE_KEY'] = "ecf52e62471efd48b23428753fe727c3"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()  # ارفعي kaggle.json الجديد
!mkdir -p /root/.kaggle
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d alessiocorrado99/animals10 -p /content --unzip

Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
License(s): GPL-2.0
 94% 551M/586M [00:06<00:00, 102MB/s] 
100% 586M/586M [00:06<00:00, 94.5MB/s]


In [ ]:
dataset_path = "/content/raw-img"
print("Dataset ready at:", dataset_path)

Dataset ready at: /content/raw-img


In [ ]:
# إنشاء مجلد المشروع داخل Drive
!mkdir -p "/content/drive/MyDrive/species_project/dataset"

# نسخ البيانات من raw-img إلى Drive
!cp -r /content/raw-img "/content/drive/MyDrive/species_project/dataset/"


In [ ]:
dataset_path = "/content/drive/MyDrive/species_project/dataset/raw-img"
print("Dataset ready at:", dataset_path)


Dataset ready at: /content/drive/MyDrive/species_project/dataset/raw-img


In [ ]:
# ====== تثبيت المكاتب المطلوبة ======
!pip install -q tensorflow scikit-learn wikipedia-api gtts

# ====== توصيل Google Drive ======
from google.colab import drive
drive.mount('/content/drive', force_remount=True)     #يضمن إعادة التوصيل في كل مرة لتجنب المشاكل.

import os, joblib, numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image as keras_image
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from IPython.display import display, Audio
from PIL import Image
import wikipediaapi
from gtts import gTTS
from google.colab import files
import cv2

# ====== إعداد المسارات ======
DATASET_PATH = "/content/drive/MyDrive/species_project/dataset/raw-img"
MODEL_PATH = "/content/drive/MyDrive/species_project/svc_pipeline_mobilenet.joblib"

# ====== قائمة الفئات ======
categories = sorted(os.listdir(DATASET_PATH))

# ====== إعداد MobileNetV2 كـ feature extractor ======
feature_extractor = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(96,96,3))

# ====== تحميل البيانات واستخراج الخصائص ======
X, y = [], []
print("جارٍ تحميل البيانات واستخراج الخصائص...")
for idx, cat in enumerate(categories):
    cat_folder = os.path.join(DATASET_PATH, cat)
    for f in os.listdir(cat_folder)[:400]:  # 400 صورة لكل فئة لتسريع التدريب
        img_path = os.path.join(cat_folder, f)
        try:
            img = keras_image.load_img(img_path, target_size=(96,96))
            x = keras_image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            feat = feature_extractor.predict(x, verbose=0)
            X.append(feat.flatten())
            y.append(idx)
        except:
            continue

X = np.array(X)
y = np.array(y)
print("✅ تم تحميل البيانات: ", X.shape)

# ====== تقسيم البيانات ======
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ====== تدريب SVM ======
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

clf = SVC(probability=True)
clf.fit(X_train_scaled, y_train)

# ====== حفظ النموذج ======
joblib.dump({'model': clf, 'scaler': scaler, 'labels': categories}, MODEL_PATH)
print("✅ تم إنشاء وحفظ النموذج:", MODEL_PATH)

# ====== قاموس ترجمة الفئات ======
label_map_ar = {
    "cane": "كلب", "cavallo": "حصان", "elefante": "فيل", "farfalla": "فراشة",
    "gallina": "دجاجة", "gatto": "قطة", "mucca": "بقرة", "pecora": "خروف",
    "ragno": "عنكبوت", "scoiattolo": "سنجاب"
}
label_map_en = {
    "cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly",
    "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep",
    "ragno": "spider", "scoiattolo": "squirrel"
}

# ====== إعداد Wikipedia API ======
wiki_ar = wikipediaapi.Wikipedia(language='ar', user_agent='MyColabSpeciesProjectBot/1.0 (contact: your_email@example.com)')
wiki_en = wikipediaapi.Wikipedia(language='en', user_agent='MyColabSpeciesProjectBot/1.0 (contact: your_email@example.com)')

def fetch_summary_for_label(orig_label, max_chars=400):
    # جلب ملخص من ويكيبيديا
    for lang, wiki, label_map in [('ar', wiki_ar, label_map_ar), ('en', wiki_en, label_map_en)]:
        candidate = label_map.get(orig_label, orig_label)
        try:
            p = wiki.page(candidate)
            if p.exists():
                return p.summary[:max_chars], lang
        except:
            continue
    return "لا توجد معلومات متاحة.", 'ar'

def preprocess_img_file(path):
    # تعديل حجم الصورة وتجهيزها للموديل
    img = keras_image.load_img(path, target_size=(96,96))
    x = keras_image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feat = feature_extractor.predict(x, verbose=0)
    return scaler.transform(feat), img

# ====== تجربة الصور ======
print("اختر/اختاري صورة (أو صور) من جهازك:")
uploaded = files.upload()

for fname in uploaded.keys():
    try:
        print("\n========================")
        print("الصورة:", fname)
        display(Image.open(fname))
        X_in, img_disp = preprocess_img_file(fname)
        probs = clf.predict_proba(X_in)[0]
        idx = int(probs.argmax())
        orig_label = categories[idx]
        conf = float(probs[idx])
        arabic_label = label_map_ar.get(orig_label, orig_label)
        print(f"التصنيف: {arabic_label} (اسم داخلي: {orig_label}) — الثقة: {conf:.3f}")

        summary, lang = fetch_summary_for_label(orig_label)
        print("ملخص ويكيبيديا:", summary)

        # تحويل النص إلى صوت
        tts_lang = 'ar' if lang=='ar' else 'en'
        try:
            tts = gTTS(text=summary, lang=tts_lang)
            out_path = f"/content/speech_{fname}.mp3"
            tts.save(out_path)
            display(Audio(out_path, autoplay=False))
            print("✅ تم إنشاء وتشغيل ملف الصوت.")
        except Exception as e:
            print("فشل تحويل النص إلى صوت:", e)
    except Exception as e:
        print("فشل معالجة الصورة:", fname, "| الخطأ:", e)


In [ ]:
# ====== خلية Colab جاهزة لتشغيل واجهة Gradio بسهولة ======
!pip install -q tensorflow scikit-learn wikipedia-api gtts gradio

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# مسارات
DRIVE_PROJECT = "/content/drive/MyDrive/species_project/"
LOCAL = "/content/species_project_local/"
import os, shutil, time
os.makedirs(LOCAL, exist_ok=True)

# انسخ الملفات المهمة من Drive إلى المسار المحلي (تأكد أن الأسماء صحيحة)
shutil.copy(os.path.join(DRIVE_PROJECT, "svc_pipeline_mobilenet.joblib"), LOCAL)
if os.path.exists(os.path.join(DRIVE_PROJECT, "sounds")):
    shutil.copytree(os.path.join(DRIVE_PROJECT, "sounds"), os.path.join(LOCAL, "sounds"), dirs_exist_ok=True)
# انسخ ملف الكود إن وُجد (اختياري)
if os.path.exists(os.path.join(DRIVE_PROJECT, "app_streamlit.py")):
    shutil.copy(os.path.join(DRIVE_PROJECT, "app_streamlit.py"), LOCAL)

# ====== كود واجهة Gradio ======
import joblib, numpy as np, gradio as gr, wikipediaapi, tempfile
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from gtts import gTTS
from PIL import Image
import pathlib

# تحميل النموذج والـ scaler والفئات
data = joblib.load(os.path.join(LOCAL, "svc_pipeline_mobilenet.joblib"))
clf = data['model']
scaler = data['scaler']
categories = data['labels']

# إعداد feature extractor (قد يحمل الأوزان من النت أول مرة)
feature_extractor = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(96,96,3))

# wikipedia
wiki_ar = wikipediaapi.Wikipedia(language='ar', user_agent='AnimalApp (AI Project by Doaa)')
wiki_en = wikipediaapi.Wikipedia(language='en', user_agent='AnimalApp (AI Project by Doaa)')

# ترجمة الفئات
label_map_ar = {
    "cane": "كلب", "cavallo": "حصان", "elefante": "فيل", "farfalla": "فراشة",
    "gallina": "دجاجة", "gatto": "قطة", "mucca": "بقرة", "pecora": "خروف",
    "ragno": "عنكبوت", "scoiattolo": "سنجاب"
}
label_map_en = {
    "cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly",
    "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep",
    "ragno": "spider", "scoiattolo": "squirrel"
}

SOUNDS_DIR = os.path.join(LOCAL, "sounds")

def fetch_summary(orig_label, lang='ar', max_chars=400):
    wiki = wiki_ar if lang=='ar' else wiki_en
    label_map = label_map_ar if lang=='ar' else label_map_en
    candidate = label_map.get(orig_label, orig_label)
    p = wiki.page(candidate)
    if p.exists():
        return p.summary[:max_chars]
    return ("لا توجد معلومات متاحة." if lang=='ar' else "No information available.")

def preprocess_img(img_pil):
    # img_pil: PIL Image
    img = img_pil.resize((96,96)).convert('RGB')
    x = np.array(img).astype('float32')
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feat = feature_extractor.predict(x, verbose=0)
    X_in = scaler.transform(feat)
    return X_in, img

def predict_and_audio(inp, language):
    # inp: file-like image (gradio passes PIL image)
    X_in, img_display = preprocess_img(inp)
    probs = clf.predict_proba(X_in)[0]
    idx = int(np.argmax(probs))
    orig_label = categories[idx]
    conf = float(probs[idx])
    # نص لواجهة
    name_ar = label_map_ar.get(orig_label, orig_label)
    name_en = label_map_en.get(orig_label, orig_label)
    display_name = name_ar if language=='عربي' else name_en

    # ملخص ويكيبيديا
    summary = fetch_summary(orig_label, 'ar' if language=='عربي' else 'en')

    # إعداد ملفات الصوت: صوت الحيوان (إن وُجد) + TTS للملخص
    animal_sound_path = None
    candidate_sound = os.path.join(SOUNDS_DIR, f"{orig_label}.mp3")
    if os.path.exists(candidate_sound):
        animal_sound_path = candidate_sound

    # TTS
    tfile = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
    try:
        tts_lang = 'ar' if language=='عربي' else 'en'
        tts = gTTS(text=summary, lang=tts_lang)
        tts.save(tfile.name)
    except Exception as e:
        tfile = None

    # نُرجع: اسم، ثقة، ملخص نصي، مسار صورة معاد تحويلها للعرض، مسارات الصوت (حقيقي ثم TTS)
    return {
        "name": display_name,
        "confidence": f"{conf:.2f}",
        "summary": summary,
        "image": img_display,
        "animal_sound": animal_sound_path,
        "tts_sound": tfile.name if tfile else None
    }

# ====== بناء واجهة Gradio Blocks بسيطة وجميلة ======
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align:center;color:#4B0082'>مشروع التعرف على الحيوانات</h1>")
    with gr.Row():
        inp = gr.Image(type="pil", label="ارفع صورة الحيوان")
        with gr.Column():
            lang = gr.Radio(['عربي','English'], value='عربي', label="اختر اللغة")
            btn = gr.Button("تصنيف")
            name_out = gr.Textbox(label="التصنيف", interactive=False)
            conf_out = gr.Textbox(label="ثقة النموذج", interactive=False)
            summary_out = gr.Textbox(label="ملخص ويكيبيديا", interactive=False)
            animal_audio = gr.Audio(label="صوت الحيوان (إن وُجد)", interactive=False)
            tts_audio = gr.Audio(label="قراءة الملخص (TTS)", interactive=False)

    def run_all(image, language):
        res = predict_and_audio(image, language)
        return res["name"], res["confidence"], res["summary"], res["animal_sound"], res["tts_sound"]

    btn.click(run_all, inputs=[inp, lang], outputs=[name_out, conf_out, summary_out, animal_audio, tts_audio])

# شغّل الواجهة مع رابط عام (share=True يعطي رابط خارجي آمن)
demo.launch(share=True, inbrowser=False)


Mounted at /content/drive
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fad5b9899b92d0ea25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
